In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import numpy as np
import json
from aml_storage import Labels, Block, Descriptor
from utils.builder import DescriptorBuilder
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from utils.hamiltonians import *
import copy
from rascal.representations import SphericalExpansion

# Ethanol

In [43]:
eth_hypers = {
    "interaction_cutoff": 5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
     "compute_gradients":  False,
}

eth_spex = RascalSphericalExpansion(eth_hypers)
mycg = ClebschGordanReal(eth_hypers["max_angular"])

In [87]:
eth_frames = ase.io.read("../data_papers/data/ethanol/dataset/ethanol_4500.xyz",":2")
water_frames = ase.io.read("../data_papers/data/water/dataset/water_randomized_1000.xyz",":2")
for f in eth_frames:
    f.cell = [100,100,100]
    f.positions += 50
for f in water_frames:
    f.cell = [100,100,100]
    f.positions += 50
    
eth_orbs = json.load(open('../data_papers/data/ethanol/dataset/orbs_saph_ethanol.json', "r"))

In [88]:
eth_rhoi = eth_spex.compute(eth_frames)

In [89]:
pairs = RascalPairExpansion(eth_hypers)
gij = pairs.compute(eth_frames)

In [90]:
def get_lm_slice(hypers):
    lm_slices = []
    start = 0
    for l in range(hypers["max_angular"] + 1):
        stop = start + 2 * l + 1
        lm_slices.append(slice(start, stop))
        start = stop
    return lm_slices

In [178]:
def rho0ij_builder(hypers, frames):
    if hypers["compute_gradients"]:
        raise Exception("Pair expansion with gradient is not implemented")
    len_frames=[len(f) for f in frames]
    max_atoms = max(len_frames)
    actual_global_species = list(
    map(int, np.unique(np.concatenate([f.numbers for f in frames]))))
    calculator = SphericalExpansion(**hypers)
    manager = calculator.transform(frames)
    info = manager.get_representation_info()
    global_species = list(range(max_atoms))
    
    hypers_ij= copy.deepcopy(hypers)
    hypers_ij["global_species"] = global_species
    hypers_ij["expansion_by_species_method"] = "user defined"
    lm_slices = get_lm_slice(hypers)

    ijframes = []
    for f in frames:
        ijf = f.copy()
        ijf.numbers = global_species[:len(f)]
        ijframes.append(ijf)

    calculator = SphericalExpansion(**hypers_ij)
    gij_expansion=[]
    for ijf in ijframes:
        gij_expansion.append(calculator.transform(ijf).get_features(calculator).reshape(len(ijf), max_atoms, hypers_ij["max_radial"], -1))
#     gij_expansion=calculator.transform(ijframes).get_features(calculator).reshape(len(ijframes), max_atoms, max_atoms, hypers_ij["max_radial"], -1) #TODO: change for differet len
#     print(gij_expansion.shape)
    
    feat_builder= DescriptorBuilder(["block_type", "L", "nu", "sigma",  "species_i", "species_j"], ["structure", "center_i", "center_j"], ["mu"], ["n"])

    pair_loc=[]
    lmax = hypers["max_angular"]
    for sp_i in actual_global_species:
        for sp_j in actual_global_species:
            center_species_mask = np.where(info[:, 2] == sp_i)[0]
            neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]
            for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
                for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
                    ifr=struct_i

                    if not (struct_i==struct_j):
                        continue
                    if atom_i==atom_j:
                        block_type = 0  # diagonal
                    elif sp_i==sp_j:
                        block_type = 1  # same-species
                    elif sp_j > sp_i:

                        block_type = 2  # different species
                    else:
                        continue

                    if [struct_i, atom_j, atom_i] not in pair_loc:
                        pair_loc.append([struct_i, atom_i, atom_j])

                    for l in range(lmax+1):
#                         print(block_type, ifr, l)
                        block_idx=(block_type, l, 0, 1, sp_i, sp_j)
                        if block_idx not in feat_builder.blocks:
                            block = feat_builder.add_block(sparse=block_idx, features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T, 
                                components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )  


                            if block_type == 1:
                                block_asym = feat_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T,
                                components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
                        else:                        
                            block = feat_builder.blocks[block_idx]
                            if block_type == 1:
                                block_asym = feat_builder.blocks[(-1,)+block_idx[1:]]

                        block_data =gij_expansion[struct_i][atom_i%len_frames[ifr], atom_j%len_frames[ifr], :, lm_slices[l]].T #TODO: change for not water
                        #POSSIBLE replacement:(atom_i-sum(info[:struct_i,:].axis=1))%len(info[np.where(info[:,:,0]==struct_i)])
                        if block_type == 1:
                            if (atom_i%len_frames[ifr])<=(atom_j%len_frames[ifr]):
                                block_data_ji = gij_expansion[struct_i][atom_j%len_frames[ifr], atom_i%len_frames[ifr], :, lm_slices[l]].T                  
                                block.add_samples(labels=np.asarray([[struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]]], dtype=np.int32), data=(block_data+block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                                block_asym.add_samples(labels=np.asarray([[struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]]], dtype=np.int32), data=(block_data-block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )

                        else:
#                             print(block_data.shape, [struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]])
                            block.add_samples(labels=np.asarray([[struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]]], dtype=np.int32), data=block_data.reshape(1, -1, block_data.shape[1]))
    #                     
    return feat_builder.build()

In [179]:
frames= [eth_frames[0],water_frames[0]]

In [180]:
eth_rho0ij=rho0ij_builder(eth_hypers, frames)

In [184]:
frames[0]

Atoms(symbols='C2OH6', pbc=False, cell=[100.0, 100.0, 100.0])

In [187]:
eth_rho0ij.block(block_type=1,L=0,species_i=1, species_j=1).samples

Labels([(0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 4, 5),
        (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 5, 6), (0, 5, 7), (0, 5, 8),
        (0, 6, 7), (0, 6, 8), (0, 7, 8), (1, 1, 2)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [10]:
total_species = sorted(set(eth_rhoi.sparse['center_species']))
# total_species = list(np.sort(np.asarray(total_species)))
lmax=eth_hypers["max_angular"]
nmax=eth_hypers["max_radial"]

In [11]:
blocks = []
for l in range(lmax+1):
    for sp_i in total_species:
        for sp_k in total_species:
            n_selected = nmax#len(np.where(opt_eva[l] > sel_thresh)[0])    
            de_block = eth_rhoi.block(center_species = sp_i, neighbor_species=sp_k, spherical_harmonics_l = l)
            block = Block(
                values = de_block.values,
                samples = de_block.samples,
                components = Labels(["m"],np.asarray(range(-l,l+1), dtype=np.int32).reshape(-1,1)),
                features = Labels(["n"], np.asarray([[n] for n in range(nmax)], dtype=np.int32))
            )
            
            blocks.append( block )

In [12]:
acdc_nu1 = Descriptor(
    sparse = Labels(names=["L", "nu", "sigma","species_i", "neighbor_species"], 
                        values=np.asarray([[ l, 1, 1, sp_i, sp_k] for l in range(eth_hypers["max_angular"]+1) 
                                                        for sp_i in total_species
                                                        for sp_k in total_species], dtype=np.int32)
                                     ), 
                      blocks = blocks
                     )
#move neighbor species to features  
# acdc_nu1.sparse_to_features('neighbor_species')

In [13]:
acdc_nu1.sparse

Labels([(0, 1, 1, 1, 1), (0, 1, 1, 1, 6), (0, 1, 1, 1, 8),
        (0, 1, 1, 6, 1), (0, 1, 1, 6, 6), (0, 1, 1, 6, 8),
        (0, 1, 1, 8, 1), (0, 1, 1, 8, 6), (0, 1, 1, 8, 8),
        (1, 1, 1, 1, 1), (1, 1, 1, 1, 6), (1, 1, 1, 1, 8),
        (1, 1, 1, 6, 1), (1, 1, 1, 6, 6), (1, 1, 1, 6, 8),
        (1, 1, 1, 8, 1), (1, 1, 1, 8, 6), (1, 1, 1, 8, 8),
        (2, 1, 1, 1, 1), (2, 1, 1, 1, 6), (2, 1, 1, 1, 8),
        (2, 1, 1, 6, 1), (2, 1, 1, 6, 6), (2, 1, 1, 6, 8),
        (2, 1, 1, 8, 1), (2, 1, 1, 8, 6), (2, 1, 1, 8, 8)],
       dtype=[('L', '<i4'), ('nu', '<i4'), ('sigma', '<i4'), ('species_i', '<i4'), ('neighbor_species', '<i4')])

In [14]:
acdc_nu1.sparse_to_features("neighbor_species")

In [15]:
acdc_nu1.sparse

Labels([(0, 1, 1, 1), (0, 1, 1, 6), (0, 1, 1, 8), (1, 1, 1, 1),
        (1, 1, 1, 6), (1, 1, 1, 8), (2, 1, 1, 1), (2, 1, 1, 6),
        (2, 1, 1, 8)],
       dtype=[('L', '<i4'), ('nu', '<i4'), ('sigma', '<i4'), ('species_i', '<i4')])

In [40]:
def tensor_g_rho_nu(rho0ij, rhoinu, hypers, cg, feature_names=None):
    """ rho_ij^{nu+1} = <q|rho_i^{nu+1}; kh> <n| rho_ij^{0}; lm> cg 
    feature_names is a tuple of the form <n_rho0ij, l_rho0ij, n's in rhoi, k|
    Make sure you have transferred the species label to the feature (sparse_to_features)
    """
    nu_ij=rho0ij.sparse["nu"][0]
    
    # rhoinu = acdc_nu1; 
    feature_names=None
    nu_ij=rho0ij.sparse["nu"][0] #should be 0
    nu_rho= rhoinu.sparse["nu"][0]
    NU = nu_rho+nu_ij

    lmax= hypers["max_angular"]

    if cg is None:
        cg = ClebschGordanReal(lmax)

    sparse_labels=copy.deepcopy(rho0ij.sparse)
    sparse_labels["nu"] = NU

    new_sparse_labels = []
    for i in sparse_labels:
        new_sparse_labels.append(tuple(i))
        i[3]*=-1
        if i not in new_sparse_labels:
            new_sparse_labels.append(tuple(i))

    X_blocks = {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}
    X_idx = {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}
    X_samples= {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}


    if feature_names is None:
        feature_names = (
            tuple(n + "_a" for n in rho0ij.block(0).features.names)
            + ("l_" + str(NU),)
            + tuple(n + "_b" for n in acdc_nu1.block(0).features.names)
            + ("k_" + str(NU),)
        )

    for index_a, block_a in rho0ij:
        block_type = index_a["block_type"]
        lam_a = index_a["L"]
        sigma_a = index_a["sigma"]
        sp_i, sp_j = index_a["species_i"], index_a["species_j"]
        for index_b, block_b in rhoinu:
            lam_b = index_b["L"]
            sigma_b = index_b["sigma"]
            rho_sp_i= index_b["species_i"]
            if not(sp_i== rho_sp_i):
                continue
            for L in range(np.abs(lam_a - lam_b),  min(lam_a + lam_b, lmax)+1):
                S = sigma_a * sigma_b * (-1) ** (lam_a + lam_b + L)
                block_idx=(block_type, L, NU, S, sp_i, sp_j)
                sel_feats=[]
                for n_a in range(len(block_a.features)):
                    f_a = tuple(block_a.features[n_a]) #values of n_a'th feature in block_a.features
                    for n_b in range(len(block_b.features)):
                        f_b = tuple(block_b.features[n_b]) #values of n_b'th feature in block_b.features
                        IDX = f_a  + (lam_a,)+f_b + (lam_b,)
                        sel_feats.append([n_a, n_b])
                        X_idx[block_idx].append(IDX)

                sel_feats = np.asarray(sel_feats, dtype=int)
                if len(sel_feats) == 0:
                    print(IDX, L, "len_feats 0")
                    continue

    #             #REMEMBER- values.shape = (nstruct*nat fitting the block criteria, 2*l+1, featsize)
                if block_type==0:
                    if not(sp_i== rho_sp_i):
                        continue
                    one_shot_blocks = cg.combine_einsum(
                     block_a.values[:, :, sel_feats[:, 0]],  #sel_feats[:,0]= n_a
                     block_b.values[:, :, sel_feats[:, 1]],  #sel_feats[:,1]= n_b*nspecies
                    L,
                    combination_string="iq,iq->iq",
                ) 

                    samples = block_a.samples
                
                else:
                    try:
                        xx=[]
                        yy=[]
                        for samplea in block_a.samples:
                            centeri = samplea['center_i']
                            centerj = samplea['center_j']
                            stra = samplea['structure']
                            idxb= block_b.samples[np.where(block_b.samples['center']==centeri)]
                            samples_idxb=block_b.samples[np.where(np.in1d(block_b.samples,idxb))[0]]
                            xx.append(samples_idxb[np.where(samples_idxb['structure']==stra)[0]])
                            if (block_type==1 or block_type==-1):
                                idxbj= block_b.samples[np.where(block_b.samples['center']==centerj)]
                                samples_idxbj=block_b.samples[np.where(np.in1d(block_b.samples,idxbj))[0]]
                                yy.append(samples_idxbj[np.where(samples_idxbj['structure']==stra)[0]])
                            iyy=[np.where(i==block_b.samples)[0][0] for i in yy]
                            rhojnuvalues = block_b.values[iyy]
                        
                        
                        ixx=[np.where(i==block_b.samples)[0][0] for i in xx]
                        rhoinuvalues = block_b.values[ixx]
                        
                    except:
                        print('FAILED')
                        print(block_type, samplea, xx, yy, index_a, index_b)
                    

                    
                    
#                     print(index_a, index_b, len(yy))
#                         rhoinuvalues = np.asarray(rhoinuvalues).reshape(-1, rhoinuvalues.shape[-2:])
                    if block_type==1:
#                         print(block_a.values[:, :, sel_feats[:, 0]].shape, rhoinuvalues[:, :, sel_feats[:, 1]].shape, rhojnuvalues[:, :, sel_feats[:, 1]].shape)
#                         print(index_a, index_b, rhoinuvalues.shape, rhojnuvalues.shape)
                        if not(sp_i== rho_sp_i):
                            continue
                        one_shot_blocks_ij = cg.combine_einsum(
                         block_a.values[:, :, sel_feats[:, 0]], 
                         rhoinuvalues[:, :, sel_feats[:, 1]],
                        L,
                        combination_string="iq,iq->iq",
                    )

                        one_shot_blocks_ji = cg.combine_einsum(
                         block_a.values[:, :, sel_feats[:, 0]], 
                         rhojnuvalues[:, :, sel_feats[:, 1]],
                        L,
                        combination_string="iq,iq->iq",
                    )
                        samples = block_a.samples

                        one_shot_blocks = (one_shot_blocks_ij+one_shot_blocks_ji)/np.sqrt(2)
    #                     one_shot_blocks_asym = (one_shot_blocks_ij-one_shot_blocks_ji)/np.sqrt(2)
                        
                    elif block_type==-1:
                        if not(sp_i== rho_sp_i):
                            continue
                        one_shot_blocks_ij = cg.combine_einsum(
                         block_a.values[:, :, sel_feats[:, 0]], 
                         rhoinuvalues[:, :, sel_feats[:, 1]],
                        L,
                        combination_string="iq,iq->iq",
                    )

                        one_shot_blocks_ji = cg.combine_einsum(
                         block_a.values[:, :, sel_feats[:, 0]], 
                         rhojnuvalues[:, :, sel_feats[:, 1]],
                        L,
                        combination_string="iq,iq->iq",
                    )
                        samples = block_a.samples
                        one_shot_blocks = (one_shot_blocks_ij-one_shot_blocks_ji)/np.sqrt(2)

                    elif block_type==2:
                        #TODO: recheck this 
        #                     if sp_i<rho_sp_i:
        #                         continue
                        if not(sp_i== rho_sp_i):
                            continue
                        #print(sp_i, rho_sp_i, sp_j, block_a.values.shape, block_b.values.shape)
                        one_shot_blocks = cg.combine_einsum(
                         block_a.values[:, :, sel_feats[:, 0]], 
                         rhoinuvalues[:, :, sel_feats[:, 1]],
                        L,
                        combination_string="iq,iq->iq",
                    )
                        samples = block_a.samples

                    else: 
                        print(block_type)
                        print("get the block type right you idiot")


                for Q in range(len(sel_feats)):
                    (n_a, n_b) = sel_feats[Q]
                    species_k, n = block_b.features[n_b]  #how to generalize this for nu
                    IDX = (n_a,)  + (lam_a,)+(species_k,)+(n,) + (lam_b,)
                    newblock = one_shot_blocks[:, :, Q]
#                     if(IDX not in X_idx[(block_type, L, NU, S, sp_i, sp_j)]):
#                         X_idx[(block_type, L, NU, S, sp_i, sp_j)].append(IDX)
   
                    X_blocks[block_idx].append(newblock)
                    X_samples[block_idx].append(samples)
    nonzero_idx = []

    nonzero_blocks = []
    for block_idx in X_blocks:
        block_type, L, NU, S, sp_i, sp_j = block_idx
        # create blocks
        if len(X_blocks[block_idx]) == 0:
            print(block_idx, "skipped")
            continue  # skips empty blocks

        nonzero_idx.append(block_idx)
        block_data = np.moveaxis(np.asarray(X_blocks[block_idx]), 0, -1) 
        block_samples = X_samples[block_idx][0]
    
        newblock = Block(
            values=block_data,
            samples=block_samples,
            components=Labels(
                ["mu"], np.asarray(range(-L, L + 1), dtype=np.int32).reshape(-1, 1)
            ),
            features=Labels(feature_names, np.asarray(X_idx[block_idx], dtype=np.int32)),
        )

        nonzero_blocks.append(newblock)
#         print(block_idx, 'done')

    X = Descriptor(
        Labels(rho0ij.sparse.names, np.asarray(nonzero_idx, dtype=np.int32)), nonzero_blocks
    )


    return X

In [41]:
rho1ij=tensor_g_rho_nu(eth_rho0ij, acdc_nu1, eth_hypers, mycg)

/Users/jigyasa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


(0, 0, 1, -1, 1, 1) skipped
(1, 0, 1, -1, 1, 1) skipped
(-1, 0, 1, -1, 1, 1) skipped
(2, 0, 1, -1, 1, 6) skipped
(2, 0, 1, -1, 1, 8) skipped
(0, 0, 1, -1, 6, 6) skipped
(1, 0, 1, -1, 6, 6) skipped
(-1, 0, 1, -1, 6, 6) skipped
(2, 0, 1, -1, 6, 8) skipped
(0, 0, 1, -1, 8, 8) skipped


In [61]:
nu_ij=eth_rho0ij.sparse["nu"][0]

# rhoinu = acdc_nu1; 
feature_names=None
nu_ij=eth_rho0ij.sparse["nu"][0] #should be 0
nu_rho= acdc_nu1.sparse["nu"][0]
NU = nu_rho+nu_ij

lmax= eth_hypers["max_angular"]
#cg = None
#if cg is None:
#    cg = ClebschGordanReal(lmax)

sparse_labels=copy.deepcopy(eth_rho0ij.sparse)
sparse_labels["nu"] = NU

new_sparse_labels = []
for i in sparse_labels:
    new_sparse_labels.append(tuple(i))
    i[3]*=-1
    if i not in new_sparse_labels:
        new_sparse_labels.append(tuple(i))

X_blocks = {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}
X_idx = {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}
X_samples= {new_sparse_labels[i]:[] for i in range(len(new_sparse_labels))}

/Users/jigyasa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


In [62]:
if feature_names is None:
    feature_names = (
        tuple(n + "_a" for n in eth_rho0ij.block(0).features.names)
        + ("l_" + str(NU),)
        + tuple(n + "_b" for n in acdc_nu1.block(0).features.names)
        + ("k_" + str(NU),)
    )

In [63]:
eth_rho0ij.sparse

Labels([( 0, 0, 0, 1, 1, 1), ( 0, 1, 0, 1, 1, 1), ( 0, 2, 0, 1, 1, 1),
        ( 1, 0, 0, 1, 1, 1), (-1, 0, 0, 1, 1, 1), ( 1, 1, 0, 1, 1, 1),
        (-1, 1, 0, 1, 1, 1), ( 1, 2, 0, 1, 1, 1), (-1, 2, 0, 1, 1, 1),
        ( 2, 0, 0, 1, 1, 6), ( 2, 1, 0, 1, 1, 6), ( 2, 2, 0, 1, 1, 6),
        ( 2, 0, 0, 1, 1, 8), ( 2, 1, 0, 1, 1, 8), ( 2, 2, 0, 1, 1, 8),
        ( 0, 0, 0, 1, 6, 6), ( 0, 1, 0, 1, 6, 6), ( 0, 2, 0, 1, 6, 6),
        ( 1, 0, 0, 1, 6, 6), (-1, 0, 0, 1, 6, 6), ( 1, 1, 0, 1, 6, 6),
        (-1, 1, 0, 1, 6, 6), ( 1, 2, 0, 1, 6, 6), (-1, 2, 0, 1, 6, 6),
        ( 2, 0, 0, 1, 6, 8), ( 2, 1, 0, 1, 6, 8), ( 2, 2, 0, 1, 6, 8),
        ( 0, 0, 0, 1, 8, 8), ( 0, 1, 0, 1, 8, 8), ( 0, 2, 0, 1, 8, 8)],
       dtype=[('block_type', '<i4'), ('L', '<i4'), ('nu', '<i4'), ('sigma', '<i4'), ('species_i', '<i4'), ('species_j', '<i4')])

In [64]:
eth_rho0ij.block(block_type=0, L=0, species_i=1, species_j=1).values.shape

(12, 1, 3)

In [28]:
acdc_nu1.sparse

Labels([(0, 1, 1, 1), (0, 1, 1, 6), (0, 1, 1, 8), (1, 1, 1, 1),
        (1, 1, 1, 6), (1, 1, 1, 8), (2, 1, 1, 1), (2, 1, 1, 6),
        (2, 1, 1, 8)],
       dtype=[('L', '<i4'), ('nu', '<i4'), ('sigma', '<i4'), ('species_i', '<i4')])

In [33]:
blockb_samples=acdc_nu1.block(L=0, species_i=1).samples
blocka_samples=eth_rho0ij.block(block_type=2, L=0, species_i=1, species_j=6).samples
blocka=eth_rho0ij.block(block_type=2, L=0, species_i=1, species_j=6)
blockb=acdc_nu1.block(L=0, species_i=1)
xx=[]
yy=[]
for samplea in blocka_samples:
    centeri = samplea['center_i']
    centerj = samplea['center_j']
    stra = samplea['structure']
    idxb= blockb_samples[np.where(blockb_samples['center']==centeri)]
    samples_idxb=blockb_samples[np.where(np.in1d(blockb_samples,idxb))[0]]
    xx.append(samples_idxb[np.where(samples_idxb['structure']==stra)[0]])
    
    idxbj= blockb_samples[np.where(blockb_samples['center']==centerj)]
    samples_idxbj=blockb_samples[np.where(np.in1d(blockb_samples,idxbj))[0]]
    yy.append(samples_idxbj[np.where(samples_idxbj['structure']==stra)[0]])
#     yy.append()
ixx=[np.where(i==blockb_samples)[0][0] for i in xx]
iyy=[np.where(i==blockb_samples)[0][0] for i in yy]

rhoinuvalues = blockb.values[ixx]
rhojnuvalues = blockb.values[iyy]

/Users/jigyasa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [36]:
blocka_samples

Labels([(0, 3, 0), (0, 3, 1), (0, 4, 0), (0, 4, 1), (0, 5, 0), (0, 5, 1),
        (0, 6, 0), (0, 6, 1), (0, 7, 0), (0, 7, 1), (0, 8, 0), (0, 8, 1),
        (1, 3, 0), (1, 3, 1), (1, 4, 0), (1, 4, 1), (1, 5, 0), (1, 5, 1),
        (1, 6, 0), (1, 6, 1), (1, 7, 0), (1, 7, 1), (1, 8, 0), (1, 8, 1)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [37]:
blockb_samples

Labels([(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 3), (1, 4),
        (1, 5), (1, 6), (1, 7), (1, 8)],
       dtype=[('structure', '<i4'), ('center', '<i4')])

In [256]:
rhojnuvalues.shape

(30, 1, 9)

In [248]:
blocka_samples

Labels([(0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 4, 5),
        (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 5, 6), (0, 5, 7), (0, 5, 8),
        (0, 6, 7), (0, 6, 8), (0, 7, 8), (1, 3, 4), (1, 3, 5), (1, 3, 6),
        (1, 3, 7), (1, 3, 8), (1, 4, 5), (1, 4, 6), (1, 4, 7), (1, 4, 8),
        (1, 5, 6), (1, 5, 7), (1, 5, 8), (1, 6, 7), (1, 6, 8), (1, 7, 8)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [24]:
blockb_samples=acdc_nu1.block(L=0, species_i=1).samples
blocka_samples=eth_rho0ij.block(block_type=2, L=0, species_i=1, species_j=6).samples
blocka=eth_rho0ij.block(block_type=1, L=0, species_i=1, species_j=1)
blockb=acdc_nu1.block(L=0, species_i=1)
# rhoinuvalues=[]
# rhojnuvalues=[]

yy=[]
xx=[]
iyy=[]
for cent in list(sorted(set(blocka_samples['center_i'])))[:]:#ordered set 
    idxa= blocka_samples[np.where(blocka_samples['center_i']==cent)]
    centj=[x[-1] for x in idxa]
#     print(cent, centj)
    idxb= blockb_samples[np.where(blockb_samples['center']==cent)]
    samples_idxb=blockb_samples[np.where(np.in1d(blockb_samples,idxb))[0]]
    
    idxbj= [blockb_samples[np.where(blockb_samples['center']==cj)] for cj in centj]
    samples_idxbj=blockb_samples[np.where(np.in1d(blockb_samples,idxbj))[0]]
    print(centj, idxbj)

    for i in range(len(idxa)):
#         print(cent, idxa[i], len(idxa))
        xx.append(samples_idxb[np.where(samples_idxb['structure']==idxa[i]['structure'])[0]])
        yy.append(samples_idxbj[np.where(samples_idxbj['structure']==idxa[i]['structure'])[0]])
        
#         iyy.append(np.where(np.in1d(samples_idxbj[np.where(samples_idxbj['structure']==idxa[i]['structure'])], blockb_samples))[0])
#         rhoinuvalues.append(blockb_samples[np.where(blockb_samples[np.where(np.in1d(blockb_samples,idxb))[0]]['structure']==idxa[i]['structure'])])
#         rhojnuvalues.append(blockb_samples[np.where(blockb_samples[np.where(np.in1d(blockb_samples,idxbj))[0]]['structure']==idxa[i]['structure'])[0]])
ixx=[np.where(i==blockb_samples)[0][0] for i in xx]
# iyy=[np.where(i==blockb_samples)[0][0] for i in yy]

rhoinuvalues = blockb.values[ixx]
# rhojnuvalues = blockb.values[iyy]
# 
#     idxj=[blockb_samples[np.where(blockb_samples['center']==y)] for y in centj]
#     idxj.apppend()
#     rhojnuvalues.append(blockb_samples[np.where(blockb_samples==y)] for y in idxj)

[0, 1, 0, 1] [Labels([], dtype=[('structure', '<i4'), ('center', '<i4')]), Labels([], dtype=[('structure', '<i4'), ('center', '<i4')]), Labels([], dtype=[('structure', '<i4'), ('center', '<i4')]), Labels([], dtype=[('structure', '<i4'), ('center', '<i4')])]


NameError: name 'xx' is not defined

In [221]:
rhoinuvalues.shape

(30, 1, 9)

In [240]:
yy

[Labels([(0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(1, 4), (1, 5), (1, 6), (1, 7), (1, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(1, 4), (1, 5), (1, 6), (1, 7), (1, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(1, 4), (1, 5), (1, 6), (1, 7), (1, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(1, 4), (1, 5), (1, 6), (1, 7), (1, 8)],
        dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(1, 4), (1, 5), (1,

In [236]:
samples_idxbj

Labels([(0, 8), (1, 8)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [228]:
blocka.samples

Labels([(0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 4, 5),
        (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 5, 6), (0, 5, 7), (0, 5, 8),
        (0, 6, 7), (0, 6, 8), (0, 7, 8), (1, 3, 4), (1, 3, 5), (1, 3, 6),
        (1, 3, 7), (1, 3, 8), (1, 4, 5), (1, 4, 6), (1, 4, 7), (1, 4, 8),
        (1, 5, 6), (1, 5, 7), (1, 5, 8), (1, 6, 7), (1, 6, 8), (1, 7, 8)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [224]:
iyy=[np.where(i==blockb_samples)[0][0] for i in yy]

30

In [231]:
centj

[8, 8]

In [178]:
 np.where(blockb_samples[np.where(np.in1d(blockb_samples,idxbj))[0]]

SyntaxError: unexpected EOF while parsing (<ipython-input-178-d48dff41b907>, line 1)

In [172]:
rhojnuvalues.shape

(30,)

In [182]:
print(cent,idxa, centj, yy)

7 [(0, 7, 8) (1, 7, 8)] [8, 8] [Labels([(0, 8)], dtype=[('structure', '<i4'), ('center', '<i4')]), Labels([(1, 8)], dtype=[('structure', '<i4'), ('center', '<i4')])]


In [189]:
samples_idxbj#[iyy[0]]

Labels([(0, 8), (1, 8)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [193]:
blockb.values

TypeError: 'numpy.ndarray' object is not callable

In [198]:
yy

[Labels([(0, 8)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(1, 8)], dtype=[('structure', '<i4'), ('center', '<i4')])]

In [199]:
blockb_samples

Labels([(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 3), (1, 4),
        (1, 5), (1, 6), (1, 7), (1, 8)],
       dtype=[('structure', '<i4'), ('center', '<i4')])

In [181]:
idxbj

[Labels([(0, 8), (1, 8)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 8), (1, 8)], dtype=[('structure', '<i4'), ('center', '<i4')])]

In [150]:
centj

[8, 8]

In [147]:
len(rhoinuvalues)

30

In [142]:
np.where(blockb_samples[np.where(np.in1d(blockb_samples,idxb))]['structure']==idxa[1]['structure'])

(array([1]),)

In [143]:
idxa

Labels([(0, 7, 8), (1, 7, 8)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [144]:
ixx

[(array([0]),),
 (array([0]),),
 (array([0]),),
 (array([0]),),
 (array([0]),),
 (array([1]),),
 (array([1]),),
 (array([1]),),
 (array([1]),),
 (array([1]),),
 (array([0]),),
 (array([0]),),
 (array([0]),),
 (array([0]),),
 (array([1]),),
 (array([1]),),
 (array([1]),),
 (array([1]),),
 (array([0]),),
 (array([0]),),
 (array([0]),),
 (array([1]),),
 (array([1]),),
 (array([1]),),
 (array([0]),),
 (array([0]),),
 (array([1]),),
 (array([1]),),
 (array([0]),),
 (array([1]),)]

In [119]:
blockb.values.shape

(12, 1, 9)

In [120]:
blockb.samples

Labels([(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 3), (1, 4),
        (1, 5), (1, 6), (1, 7), (1, 8)],
       dtype=[('structure', '<i4'), ('center', '<i4')])

In [122]:
xx

[Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')])]

In [115]:
blockb.values.shape

(12, 1, 9)

In [112]:
rhoinuvalues

[Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), (

In [101]:
blockb_samples[np.where((np.in1d(blockb_samples,idxb)) & (np.in1d(idxa[i]['structure'],idxb['structure'])))]

Labels([(0, 7), (1, 7)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [105]:
blockb_samples[np.where(blockb_samples[np.where(np.in1d(blockb_samples,idxb))]['structure']==idxa[i]['structure'])]


Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [109]:
xx

[Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')]),
 Labels([(0, 4)], dtype=[('structure', '<i4'), ('center', '<i4')])]

In [82]:
blockb_samples[np.where(np.in1d(idxa[i]['structure'],idxb['structure']))[0]]

Labels([(0, 3)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [78]:
xx[0]

Labels([(0, 7), (1, 7)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [72]:
idxa

Labels([(0, 7, 8), (1, 7, 8)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [24]:
idxb

Labels([(0, 7), (1, 7)], dtype=[('structure', '<i4'), ('center', '<i4')])

In [66]:
len(rhoinuvalues)

30

In [75]:
blocka.values[np.where(blocka_samples==idxa)].shape

/tmp/ipykernel_267857/4199430762.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  blocka.values[np.where(blocka_samples==idxa)].shape


(0, 1, 3)

In [81]:
idxa
blocka_samples

Labels([(0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 4, 5),
        (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 5, 6), (0, 5, 7), (0, 5, 8),
        (0, 6, 7), (0, 6, 8), (0, 7, 8)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [77]:
np.where(blocka_samples==idxa)

/tmp/ipykernel_267857/115395596.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.where(blocka_samples==idxa)


(array([], dtype=int64),)

In [69]:
blocka_samples

Labels([(0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 4, 5),
        (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 5, 6), (0, 5, 7), (0, 5, 8),
        (0, 6, 7), (0, 6, 8), (0, 7, 8)],
       dtype=[('structure', '<i4'), ('center_i', '<i4'), ('center_j', '<i4')])

In [78]:
np.where(blockb_samples==idxb)[0]

array([0])

In [66]:
blockb_samples

Labels([(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
       dtype=[('structure', '<i4'), ('center', '<i4')])

In [68]:
blockb= acdc_nu1.block(L=0, species_i=1)
blockb.values[np.where(blockb_samples==idxb)[0]]

array([[[ 1.08908493e-02,  1.35511093e-03,  1.32078856e-03,
          4.61973895e-03,  1.01437912e-03, -1.67138313e-04,
          8.66725818e-05,  1.26724882e-03,  3.89070589e-04]]])

In [48]:
blockb_samples=acdc_nu1.block(L=0, species_i=1).samples
blockb_samples[np.where(blockb_samples['center']==3)]

Labels([(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)],
       dtype=[('structure', '<i4'), ('center', '<i4')])

In [39]:
for index_a, block_a in eth_rho0ij:
    block_type = index_a["block_type"]
    lam_a = index_a["L"]
    sigma_a = index_a["sigma"]
    sp_i, sp_j = index_a["species_i"], index_a["species_j"]
    for index_b, block_b in acdc_nu1:
        lam_b = index_b["L"]
        sigma_b = index_b["sigma"]
        rho_sp_i= index_b["species_i"]
        if not(sp_i== rho_sp_i):
            continue
        for L in range(np.abs(lam_a - lam_b),  min(lam_a + lam_b, lmax)+1):
            S = sigma_a * sigma_b * (-1) ** (lam_a + lam_b + L)
            block_idx=(block_type, L, NU, S, sp_i, sp_j)
            sel_feats=[]
            for n_a in range(len(block_a.features)):
                f_a = tuple(block_a.features[n_a]) #values of n_a'th feature in block_a.features
                for n_b in range(len(block_b.features)):
                    f_b = tuple(block_b.features[n_b]) #values of n_b'th feature in block_b.features
                    IDX = f_a  + (lam_a,)+f_b + (lam_b,)
                    sel_feats.append([n_a, n_b])
                    X_idx[block_idx].append(IDX)

            sel_feats = np.asarray(sel_feats, dtype=int)
            if len(sel_feats) == 0:
                print(IDX, L, "len_feats 0")
                continue
            
            #REMEMBER- values.shape = (nstruct*nat fitting the block criteria, 2*l+1, featsize)
            if block_type==0:
                if not(sp_i== rho_sp_i):
                    continue
                one_shot_blocks = cg.combine_einsum(
                 block_a.values[:, :, sel_feats[:, 0]],  #sel_feats[:,0]= n_a
                 block_b.values[:, :, sel_feats[:, 1]],  #sel_feats[:,1]= n_b*nspecies
                L,
                combination_string="iq,iq->iq",
            ) 

                samples = block_a.samples

            elif block_type==1: #or block_type==-1: 
                #print(block_a.values[:, :, sel_feats[:, 0]].shape, block_b.values[0::2, :, sel_feats[:, 0]].shape)
                if not(sp_i== rho_sp_i):
                    continue
                one_shot_blocks_ij = cg.combine_einsum(
                 block_a.values[:, :, sel_feats[:, 0]], 
                 block_b.values[0::2, :, sel_feats[:, 1]],
                L,
                combination_string="iq,iq->iq",
            )

#                 one_shot_blocks_ji = cg.combine_einsum(
#                  block_a.values[:, :, sel_feats[:, 0]], 
#                  block_b.values[1::2, :, sel_feats[:, 1]],
#                 L,
#                 combination_string="iq,iq->iq",
#             )
#                 samples = block_a.samples

#                 one_shot_blocks = (one_shot_blocks_ij+one_shot_blocks_ji)/np.sqrt(2)
# #                     one_shot_blocks_asym = (one_shot_blocks_ij-one_shot_blocks_ji)/np.sqrt(2)
                

#             elif block_type==-1:
#                 if not(sp_i== rho_sp_i):
#                     continue
#                 one_shot_blocks_ij = cg.combine_einsum(
#                  block_a.values[:, :, sel_feats[:, 0]], 
#                  block_b.values[0::2, :, sel_feats[:, 1]],
#                 L,
#                 combination_string="iq,iq->iq",
#             )

#                 one_shot_blocks_ji = cg.combine_einsum(
#                  block_a.values[:, :, sel_feats[:, 0]], 
#                  block_b.values[1::2, :, sel_feats[:, 1]],
#                 L,
#                 combination_string="iq,iq->iq",
#             )
#                 samples = block_a.samples
#                 one_shot_blocks = (one_shot_blocks_ij-one_shot_blocks_ji)/np.sqrt(2)

#             elif block_type==2:
#                 #TODO: recheck this 
# #                     if sp_i<rho_sp_i:
# #                         continue
#                 if not(sp_i== rho_sp_i):
#                     continue
#                 #print(sp_i, rho_sp_i, sp_j, block_a.values.shape, block_b.values.shape)
#                 one_shot_blocks = cg.combine_einsum(
#                  block_a.values[:, :, sel_feats[:, 0]], 
#                  block_b.values[:, :, sel_feats[:, 1]],
#                 L,
#                 combination_string="iq,iq->iq",
#             )
#                 samples = block_a.samples

#             else: 
#                 print(block_type)
#                 print("get the block type right you idiot")

#             for Q in range(len(sel_feats)):
#                 (n_a, n_b) = sel_feats[Q]
#                 species_k, n = block_b.features[n_b]  #how to generalize this for nu
#                 IDX = (n_a,)  + (lam_a,)+(species_k,)+(n,) + (lam_b,)
#                 newblock = one_shot_blocks[:, :, Q]
# #                     if(IDX not in X_idx[(block_type, L, NU, S, sp_i, sp_j)]):
# #                         X_idx[(block_type, L, NU, S, sp_i, sp_j)].append(IDX)

#                 X_blocks[block_idx].append(newblock)
#                 X_samples[block_idx].append(samples)

IndexError: Cannot combine feature blocks with different number of items

In [28]:
print(sel_feats)

[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [0 7]
 [0 8]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [1 4]
 [1 5]
 [1 6]
 [1 7]
 [1 8]
 [2 0]
 [2 1]
 [2 2]
 [2 3]
 [2 4]
 [2 5]
 [2 6]
 [2 7]
 [2 8]]


In [31]:
l = []
for i in range(len(eth_rho0ij.block(0).features)):
    #print(tuple(rho0ij.block(0).features[i]))
    for j in range(len(acdc_nu1.block(0).features)):
        #print(tuple(acdc_nu1.block(0).features[j]))
        l.append([i,j])
print(l)
  

[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8]]


In [30]:
acdc_nu1.block(0).features

Labels([(1, 0), (1, 1), (1, 2), (6, 0), (6, 1), (6, 2), (8, 0), (8, 1),
        (8, 2)], dtype=[('neighbor_species', '<i4'), ('n', '<i4')])

In [26]:
block_a.values.shape

(1, 5, 3)

In [27]:
block_b.values.shape

(1, 5, 9)

In [153]:
for i,j in enumerate(eth_rho0ij.sparse):
    print(i, j, eth_rho0ij.block(i).values.shape)

0 (0, 0, 0, 1, 1, 1) (6, 1, 3)
1 (0, 1, 0, 1, 1, 1) (6, 3, 3)
2 (0, 2, 0, 1, 1, 1) (6, 5, 3)
3 (1, 0, 0, 1, 1, 1) (15, 1, 3)
4 (-1, 0, 0, 1, 1, 1) (15, 1, 3)
5 (1, 1, 0, 1, 1, 1) (15, 3, 3)
6 (-1, 1, 0, 1, 1, 1) (15, 3, 3)
7 (1, 2, 0, 1, 1, 1) (15, 5, 3)
8 (-1, 2, 0, 1, 1, 1) (15, 5, 3)
9 (2, 0, 0, 1, 1, 6) (12, 1, 3)
10 (2, 1, 0, 1, 1, 6) (12, 3, 3)
11 (2, 2, 0, 1, 1, 6) (12, 5, 3)
12 (2, 0, 0, 1, 1, 8) (6, 1, 3)
13 (2, 1, 0, 1, 1, 8) (6, 3, 3)
14 (2, 2, 0, 1, 1, 8) (6, 5, 3)
15 (0, 0, 0, 1, 6, 6) (2, 1, 3)
16 (0, 1, 0, 1, 6, 6) (2, 3, 3)
17 (0, 2, 0, 1, 6, 6) (2, 5, 3)
18 (1, 0, 0, 1, 6, 6) (1, 1, 3)
19 (-1, 0, 0, 1, 6, 6) (1, 1, 3)
20 (1, 1, 0, 1, 6, 6) (1, 3, 3)
21 (-1, 1, 0, 1, 6, 6) (1, 3, 3)
22 (1, 2, 0, 1, 6, 6) (1, 5, 3)
23 (-1, 2, 0, 1, 6, 6) (1, 5, 3)
24 (2, 0, 0, 1, 6, 8) (2, 1, 3)
25 (2, 1, 0, 1, 6, 8) (2, 3, 3)
26 (2, 2, 0, 1, 6, 8) (2, 5, 3)
27 (0, 0, 0, 1, 8, 8) (1, 1, 3)
28 (0, 1, 0, 1, 8, 8) (1, 3, 3)
29 (0, 2, 0, 1, 8, 8) (1, 5, 3)


In [194]:
for i,j in enumerate(acdc_nu1.sparse):
    print(i, j, acdc_nu1.block(i).values.shape)

0 (0, 1, 1, 1) (6, 1, 9)
1 (0, 1, 1, 6) (2, 1, 9)
2 (0, 1, 1, 8) (1, 1, 9)
3 (1, 1, 1, 1) (6, 3, 9)
4 (1, 1, 1, 6) (2, 3, 9)
5 (1, 1, 1, 8) (1, 3, 9)
6 (2, 1, 1, 1) (6, 5, 9)
7 (2, 1, 1, 6) (2, 5, 9)
8 (2, 1, 1, 8) (1, 5, 9)


In [34]:
eth_rho0ij.block(3).values

array([[[ 2.89158690e-03,  1.41707457e-03, -3.13228277e-04]],

       [[ 7.05859447e-04,  2.19100994e-03,  1.62279635e-04]],

       [[ 5.13452393e-04,  2.13343517e-03,  2.60410483e-04]],

       [[ 5.57814491e-07,  1.52427094e-03,  7.00540139e-04]],

       [[ 1.31868125e-03,  2.16389036e-03, -5.87824234e-05]],

       [[ 5.71533828e-04,  2.15569722e-03,  2.28730151e-04]],

       [[ 2.42691026e-05,  1.59152070e-03,  6.66026516e-04]],

       [[ 5.07001057e-04,  2.13066327e-03,  2.64053973e-04]],

       [[ 1.58229242e-03,  2.09014757e-03, -1.25803505e-04]],

       [[ 3.52775687e-03,  9.66674657e-04, -3.44020896e-04]],

       [[ 2.53802333e-03,  1.63775160e-03, -2.81445327e-04]],

       [[-5.77110689e-05,  3.45943409e-04,  1.06937494e-03]],

       [[ 2.57680230e-03,  1.61472161e-03, -2.85507251e-04]],

       [[-8.57699928e-05,  6.29281266e-04,  1.02151383e-03]],

       [[-9.09245769e-05,  7.35916759e-04,  9.95818222e-04]]])

In [191]:
x = eth_rho0ij.block(0).values[:, :, sel_feats[:, 0]]

In [192]:
y = acdc_nu1.block(0).values[:, :, sel_feats[:, 1]]

In [76]:
nonzero_idx = []

nonzero_blocks = []
for block_idx in X_blocks:
    block_type, L, NU, S, sp_i, sp_j = block_idx
    # create blocks
    if len(X_blocks[block_idx]) == 0:
        print(block_idx, "skipped")
        continue  # skips empty blocks

    nonzero_idx.append(block_idx)
    block_data = np.moveaxis(np.asarray(X_blocks[block_idx]), 0, -1) 
    block_samples = X_samples[block_idx][0]

    newblock = Block(
        values=block_data,
        samples=block_samples,
        components=Labels(
            ["mu"], np.asarray(range(-L, L + 1), dtype=np.int32).reshape(-1, 1)
        ),
        features=Labels(feature_names, np.asarray(X_idx[block_idx], dtype=np.int32)),
    )

    nonzero_blocks.append(newblock)
#         print(block_idx, 'done')

X = Descriptor(
    Labels(rho0ij.sparse.names, np.asarray(nonzero_idx, dtype=np.int32)), nonzero_blocks
)


return X

(0, 0, 1, 1, 1, 1) skipped
(0, 0, 1, -1, 1, 1) skipped
(0, 1, 1, 1, 1, 1) skipped
(0, 1, 1, -1, 1, 1) skipped
(0, 2, 1, 1, 1, 1) skipped
(0, 2, 1, -1, 1, 1) skipped
(1, 0, 1, 1, 1, 1) skipped
(1, 0, 1, -1, 1, 1) skipped
(-1, 0, 1, 1, 1, 1) skipped
(-1, 0, 1, -1, 1, 1) skipped
(1, 1, 1, 1, 1, 1) skipped
(1, 1, 1, -1, 1, 1) skipped
(-1, 1, 1, 1, 1, 1) skipped
(-1, 1, 1, -1, 1, 1) skipped
(1, 2, 1, 1, 1, 1) skipped
(1, 2, 1, -1, 1, 1) skipped
(-1, 2, 1, 1, 1, 1) skipped
(-1, 2, 1, -1, 1, 1) skipped
(2, 0, 1, 1, 1, 6) skipped
(2, 0, 1, -1, 1, 6) skipped
(2, 1, 1, 1, 1, 6) skipped
(2, 1, 1, -1, 1, 6) skipped
(2, 2, 1, 1, 1, 6) skipped
(2, 2, 1, -1, 1, 6) skipped
(2, 0, 1, 1, 1, 8) skipped
(2, 0, 1, -1, 1, 8) skipped
(2, 1, 1, 1, 1, 8) skipped
(2, 1, 1, -1, 1, 8) skipped
(2, 2, 1, 1, 1, 8) skipped
(2, 2, 1, -1, 1, 8) skipped
(0, 0, 1, 1, 6, 6) skipped
(0, 0, 1, -1, 6, 6) skipped
(0, 1, 1, 1, 6, 6) skipped
(0, 1, 1, -1, 6, 6) skipped
(0, 2, 1, 1, 6, 6) skipped
(0, 2, 1, -1, 6, 6) skipped
(1, 

NameError: name 'rho0ij' is not defined

# CRAP 

In [139]:
frames= [eth_frames[0],water_frames[0]]

In [156]:
hypers= eth_hypers ; 
if hypers["compute_gradients"]:
    raise Exception("Pair expansion with gradient is not implemented")
len_frames=[len(f) for f in frames]
max_atoms = max(len_frames)
actual_global_species = list(
map(int, np.unique(np.concatenate([f.numbers for f in frames]))))
calculator = SphericalExpansion(**hypers)
manager = calculator.transform(frames)
info = manager.get_representation_info()
global_species = list(range(max_atoms))

hypers_ij= copy.deepcopy(hypers)
hypers_ij["global_species"] = global_species
hypers_ij["expansion_by_species_method"] = "user defined"
lm_slices = get_lm_slice(hypers)

ijframes = []
for f in frames:
    ijf = f.copy()
    ijf.numbers = global_species[:len(f)]
    ijframes.append(ijf)

calculator = SphericalExpansion(**hypers_ij)
gij_expansion=[]
for ijf in ijframes:
    gij_expansion.append(calculator.transform(ijf).get_features(calculator).reshape(len(ijf), max_atoms, hypers_ij["max_radial"], -1))
# gij_expansion=calculator.transform(ijframes).get_features(calculator).reshape(len(ijframes), max_atoms, max_atoms, hypers_ij["max_radial"], -1) #TODO: change for differet len
#     print(gij_expansion.shape)

In [171]:
gij_expansion[1][0,2]

array([[ 4.47987330e-03,  6.91553931e-03,  0.00000000e+00,
         6.76429916e-04,  1.21706177e-03,  0.00000000e+00,
        -3.62627354e-03,  0.00000000e+00, -6.16184551e-03],
       [-1.12749759e-03, -1.68716514e-03,  0.00000000e+00,
        -1.65026750e-04, -2.80319664e-04,  0.00000000e+00,
         8.35221188e-04,  0.00000000e+00,  1.41922662e-03],
       [-1.34135165e-04, -2.18250249e-04,  0.00000000e+00,
        -2.13477202e-05, -4.18020897e-05,  0.00000000e+00,
         1.24550631e-04,  0.00000000e+00,  2.11639231e-04]])

In [161]:
feat_builder= DescriptorBuilder(["block_type", "L", "nu", "sigma",  "species_i", "species_j"], ["structure", "center_i", "center_j"], ["mu"], ["n"])

pair_loc=[]
lmax = hypers["max_angular"]
for sp_i in actual_global_species:
    for sp_j in actual_global_species:
        center_species_mask = np.where(info[:, 2] == sp_i)[0]
        neighbor_species_mask = np.where(info[:, 2] == sp_j)[0]
        for i, (struct_i, atom_i) in enumerate(info[center_species_mask[:]][...,:-1]):
            for j, (struct_j, atom_j) in enumerate(info[neighbor_species_mask[:]][...,:-1]):
                ifr=struct_i

                if not (struct_i==struct_j):
                    continue
                if atom_i==atom_j:
                    block_type = 0  # diagonal
                elif sp_i==sp_j:
                    block_type = 1  # same-species
                elif sp_j > sp_i:

                    block_type = 2  # different species
                else:
                    continue

                if [struct_i, atom_j, atom_i] not in pair_loc:
                    pair_loc.append([struct_i, atom_i, atom_j])

                for l in range(lmax+1):
                    print(block_type, ifr, l)
                    block_idx=(block_type, l, 0, 1, sp_i, sp_j)
                    if block_idx not in feat_builder.blocks:
                        block = feat_builder.add_block(sparse=block_idx, features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T, 
                            components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )  


                        if block_type == 1:
                            block_asym = feat_builder.add_block(sparse=(-1,)+block_idx[1:], features=np.asarray([list(range(hypers["max_radial"]))], dtype=np.int32).T,
                            components=np.asarray([list(range(-l, l+1))], dtype=np.int32 ).T )                                                    
                    else:                        
                        block = feat_builder.blocks[block_idx]
                        if block_type == 1:
                            block_asym = feat_builder.blocks[(-1,)+block_idx[1:]]

                    block_data =gij_expansion[struct_i][atom_i%len_frames[ifr], atom_j%len_frames[ifr], :, lm_slices[l]].T #TODO: change for not water
                    #POSSIBLE replacement:(atom_i-sum(info[:struct_i,:].axis=1))%len(info[np.where(info[:,:,0]==struct_i)])
                    if block_type == 1:
                        if (atom_i%len_frames[ifr])<=(atom_j%len_frames[ifr]):
                            block_data_ji = gij_expansion[struct_i][atom_j%len_frames[ifr], atom_i%len_frames[ifr], :, lm_slices[l]].T                  
                            block.add_samples(labels=np.asarray([[struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]]], dtype=np.int32), data=(block_data+block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                            block_asym.add_samples(labels=np.asarray([[struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]]], dtype=np.int32), data=(block_data-block_data_ji).reshape((1,-1,block_data.shape[1]))/np.sqrt(2) )
                    
                    else:
                        print(block_data.shape, [struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]])
                        block.add_samples(labels=np.asarray([[struct_i, atom_i%len_frames[ifr], atom_j%len_frames[ifr]]], dtype=np.int32), data=block_data.reshape(1, -1, block_data.shape[1]))
#                     

0 0 0
(1, 3) [0, 3, 3]
0 0 1
(3, 3) [0, 3, 3]
0 0 2
(5, 3) [0, 3, 3]
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
0 0 0
(1, 3) [0, 4, 4]
0 0 1
(3, 3) [0, 4, 4]
0 0 2
(5, 3) [0, 4, 4]
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
0 0 0
(1, 3) [0, 5, 5]
0 0 1
(3, 3) [0, 5, 5]
0 0 2
(5, 3) [0, 5, 5]
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
0 0 0
(1, 3) [0, 6, 6]
0 0 1
(3, 3) [0, 6, 6]
0 0 2
(5, 3) [0, 6, 6]
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
0 0 0
(1, 3) [0, 7, 7]
0 0 1
(3, 3) [0, 7, 7]
0 0 2
(5, 3) [0, 7, 7]
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
1 0 0
1 0 1
1 0 2
0 0 0
(1, 3) [0, 8, 8]
0 0 1
(3, 3) [0, 8, 8]
0 0 2
(5, 3) [0, 8, 8]
0 1 0
(1, 3) [1, 1, 1]
0 1 1
(3, 3) [1, 1, 1]


In [144]:
gij_expansion.shape

(2, 9, 9, 3, 6)

In [129]:
for j, (struct_j, atom_j) in enumerate(info[center_species_mask[:]][...,:-1]):
    print(j, struct_j, atom_j)

0 0 3
1 0 4
2 0 5
3 0 6
4 0 7
5 0 8
6 1 10
7 1 11


In [130]:
block_data.shape

(2, 3)

In [136]:
frames=eth_frames

In [120]:
info[center_species_mask[:]][...,:-1]

array([[ 0,  3],
       [ 0,  4],
       [ 0,  5],
       [ 0,  6],
       [ 0,  7],
       [ 0,  8],
       [ 1, 10],
       [ 1, 11]], dtype=int32)

In [113]:
gij_expansion.shape

(2, 9, 9, 3, 6)

In [114]:
block_data

array([[0., 0., 0.],
       [0., 0., 0.]])

In [106]:
block_data.reshape(1, -1, block_data.shape[1]).shape

(1, 2, 3)

In [102]:
info

array([[ 0,  0,  6],
       [ 0,  1,  6],
       [ 0,  2,  8],
       [ 0,  3,  1],
       [ 0,  4,  1],
       [ 0,  5,  1],
       [ 0,  6,  1],
       [ 0,  7,  1],
       [ 0,  8,  1],
       [ 1,  9,  8],
       [ 1, 10,  1],
       [ 1, 11,  1]], dtype=int32)

In [103]:
frames

[Atoms(symbols='C2OH6', pbc=False, cell=[100.0, 100.0, 100.0]),
 Atoms(symbols='OH2', pbc=True, cell=[100.0, 100.0, 100.0])]